In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Crown-of-Thorns Starfish Detection Pipeline

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/models/blob/master/official/projects/cots_detector/crown_of_thorns_starfish_detection_pipeline.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/models/blob/master/official/projects/cots_detector/crown_of_thorns_starfish_detection_pipeline.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View on GitHub</a>
  </td>
</table>

Coral reefs are some of the most diverse and important ecosystems in the world , however they face a number of rising threats that have resulted in massive global declines. In Australia, outbreaks of the coral-eating crown-of-thorns starfish (COTS) have been shown to cause major coral loss, with just 15 starfish in a hectare being able to strip a reef of 90% of its coral tissue. While COTS naturally exist in the Indo-Pacific, overfishing and excess run-off nutrients have led to massive outbreaks that are devastating already vulnerable coral communities.

Controlling COTS populations is critical to promoting coral growth and resilience, so Google teamed up with Australia’s national science agency, [CSIRO](https://www.csiro.au/en/), to tackle this problem. We trained ML object detection models to help scale underwater surveys, enabling the monitoring and mapping out these harmful invertebrates with the ultimate goal of helping control teams to address and prioritize outbreaks.

### This notebook

This notebook tutorial shows how to detect COTS using a pre-trained COTS detector implemented in TensorFlow. On top of just running the model on each frame of the video, the tracking code in this notebook aligns detections from frame to frame creating a consistent track for each COTS. Each track is given an id and frame count. Here is an example image from a video of a reef showing labeled COTS starfish.

<img src="https://storage.googleapis.com/download.tensorflow.org/data/cots_detection/COTS_detected_sample.png">

It is recommended to enable GPU to accelerate the inference. On CPU, this runs for about 40 minutes, but on GPU it takes only 10 minutes. (from colab menu: *Runtime > Change runtime type > Hardware accelerator > select "GPU"*).

Install all needed packages.

In [ ]:
# remove the existing datascience package to avoid package conflicts in the colab environment
!pip3 uninstall -y datascience
!pip3 install -q opencv-python

In [ ]:
# Imports
import base64
import copy
import dataclasses
import glob
import logging
import mimetypes
import os
import pathlib
import subprocess
import time
import textwrap

from absl import logging as absl_logging
from IPython import display
import cv2
import numpy as np
import tensorflow as tf
from tqdm import tqdm

Define all needed variables.

In [ ]:
model_name = "cots_1080_v1" #@param ["cots_1080_v1", "cots_720_v1"]
test_sequence_name = "test3" #@param ["test1", "test2", "test3"]

In [ ]:
cots_model = f"https://storage.googleapis.com/download.tensorflow.org/models/cots_detection/{model_name}.zip"

# Alternatively, this dataset can be downloaded through CSIRO's Data Access Portal at https://data.csiro.au/collection/csiro:54830v2
sample_data_link = f"https://storage.googleapis.com/download.tensorflow.org/data/cots_detection/sample_images.zip"

preview_video_path = "preview.mp4"
detection_small_video_path = "COTS_detection.mp4"
detection_csv_path = "detections.csv"

Also, download the trained COTS detection model that matches your preferences above.

In [ ]:
model_path = tf.keras.utils.get_file(origin=cots_model)
# Unzip model
!mkdir {model_name}
!unzip -o -q {model_path} -d {model_name}

You also need to retrieve the sample data.  This sample data is made up of a series of chronological images.

In [ ]:
sample_data_path = tf.keras.utils.get_file(origin=sample_data_link)
# Unzip data
!mkdir sample_images
!unzip -o -q {sample_data_path} -d sample_images

# Load the model and perform inference and tracking on sample data
Load trained model from disk and create the inference function `model_fn()`.  This might take a little while.

In [ ]:
absl_logging.set_verbosity(absl_logging.ERROR)

tf.config.optimizer.set_experimental_options({'auto_mixed_precision': True})
tf.config.optimizer.set_jit(True)

model_fn = tf.saved_model.load(model_name).signatures['serving_default']

# Define **OpticalFlowTracker** class and its related classes

These help track the movement of each COTS object throughout the image frames.

In [ ]:
def box_area(x0, y0, x1, y1):
    return (x1 - x0 + 1) * (y1 - y0 + 1)

@dataclasses.dataclass
class Detection:
    """Detection dataclass."""
    class_id: int
    score: float
    x0: float
    y0: float
    x1: float
    y1: float

    def __repr__(self):
        return (f'Class {self.class_id}, score {self.score}, '
                f'box ({self.x0}, {self.y0}, {self.x1}, {self.y1})')

    def area(self):
        return box_area(self.x0, self.y0, self.x1, self.y1)

    def iou(self, other):
        overlap_x0 = max(self.x0, other.x0)
        overlap_y0 = max(self.y0, other.y0)
        overlap_x1 = min(self.x1, other.x1)
        overlap_y1 = min(self.y1, other.y1)
        if overlap_x0 < overlap_x1 and overlap_y0 < overlap_y1:
            overlap_area = box_area(overlap_x0, overlap_y0, overlap_x1,
                                    overlap_y1)
            return overlap_area / (self.area() + other.area() - overlap_area)
        else:
            return 0


In [ ]:
class Tracklet:
    def __init__(self, timestamp, detection):
        self.timestamp = timestamp
        # Store a copy here to make sure the coordinates will not be updated
        # when the optical flow propagation runs using another reference to this
        # detection.
        self.detection = copy.deepcopy(detection)

    def __repr__(self):
        return f'Time {self.timestamp}, ' + self.detection.__repr__()


In [ ]:
class Track:
    """Tracker entries."""
    def __init__(self, id, detection):
        self.id = id
        self.linked_dets = []
        self.det = detection

    def __repr__(self):
        result = f'Track {self.id}'
        for linked_det in self.linked_dets:
            result += '\n' + linked_det.__repr__()
        return result


In [ ]:
class OpticalFlowTracker:
    """Optical flow tracker."""
    def __init__(self, tid, ft=3.0, iou=0.5, tt=2.0, bb=32, size=64, its=20,
                 eps=0.03, levels=3):
        self.track_id = tid
        # How long to apply optical flow tracking without getting positive 
        # detections (sec).
        self.track_flow_time = ft * 1000
        # Required IoU overlap to link a detection to a track.
        self.overlap_threshold = iou
        # Used to detect if detector needs to be reset.
        self.time_threshold = tt * 1000
        self.border = bb
        # Size of optical flow region.
        self.of_size = (size, size)
        self.of_criteria = (cv2.TermCriteria_COUNT + cv2.TermCriteria_EPS, its, 
                            eps)
        self.of_levels= levels

        self.tracks = []
        self.prev_image = None
        self.prev_time = -1

    def update(self, image_bgr, detections, timestamp):
        start = time.time()
        num_optical_flow_calls = 0

        image = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2GRAY)

        image_w = image.shape[1]
        image_h = image.shape[0]

        # Assume tracker is invalid if too much time has passed!
        if (self.prev_time > 0 and
                timestamp - self.prev_time > self.time_threshold):
            logging.info(
                'Too much time since last update, resetting tracker.')
            self.tracks = []

        # Remove tracks which are:
        # - Touching the image edge.
        # - Have existed for a long time without linking a real detection.
        active_tracks = []
        for track in self.tracks:
            if (track.det.x0 < self.border or track.det.y0 < self.border or
                    track.det.x1 >= (image_w - self.border) or
                    track.det.y1 >= (image_h - self.border)):
                logging.info(f'Removing track {track.id} because it\'s near the border')
                continue

            time_since_last_detection = timestamp - track.linked_dets[-1].timestamp
            if (time_since_last_detection > self.track_flow_time):
                logging.info(f'Removing track {track.id} because it\'s too old '
                             f'({time_since_last_detection:.02f}s)')
                continue

            active_tracks.append(track)

        self.tracks = active_tracks

        # Run optical flow to update existing tracks.
        if self.prev_time > 0:
            # print('Running optical flow propagation.')
            of_params = {
                'winSize': self.of_size,
                'maxLevel': self.of_levels,
                'criteria': self.of_criteria
            }
            for track in self.tracks:
                input_points = np.float32([[[(track.det.x0 + track.det.x1) / 2,
                                             (track.det.y0 + track.det.y1) / 2]]])
                output_points, status, error = cv2.calcOpticalFlowPyrLK(
                    self.prev_image, image, input_points, None, **of_params)
                num_optical_flow_calls += 1
                w = track.det.x1 - track.det.x0
                h = track.det.y1 - track.det.y0
                # print(f'Detection before flow update: {track.det}')
                track.det.x0 = output_points[0][0][0] - w * 0.5
                track.det.y0 = output_points[0][0][1] - h * 0.5
                track.det.x1 = output_points[0][0][0] + w * 0.5
                track.det.y1 = output_points[0][0][1] + h * 0.5
                # print(f'Detection after flow update: {track.det}')


        # Insert new detections.
        detected_obj_track_ids = set()

        for detection in detections:
            if (detection.x0 < self.border or detection.y0 < self.border or
                    detection.x1 >= image_w - self.border or
                    detection.y1 >= image_h - self.border):
                # print('Skipping detection because it\'s close to the border.')
                continue

            # See if detection can be linked to an existing track.
            linked = False
            overlap_index = 0
            overlap_max = -1000
            for track_index, track in enumerate(self.tracks):
                # print(f'Testing track {track_index}')
                if track.det.class_id != detection.class_id:
                    continue
                overlap = detection.iou(track.det)
                if overlap > overlap_max:
                    overlap_index = track_index
                    overlap_max = overlap

            # Link to existing track with maximal IoU.
            if overlap_max > self.overlap_threshold:
                track = self.tracks[overlap_index]
                track.det = detection
                track.linked_dets.append(Tracklet(timestamp, detection))
                detected_obj_track_ids.add(track.id)
                linked = True

            if not linked:
                logging.info(f'Creating new track with ID {self.track_id}')
                new_track = Track(self.track_id, detection)
                new_track.linked_dets.append(Tracklet(timestamp, detection))
                detected_obj_track_ids.add(self.track_id)
                self.tracks.append(new_track)
                self.track_id += 1

        for track in self.tracks:
            # If the detector does not find the obj but estimated in the tracker, 
            # add the estimated one to that tracker's linked_dets
            if track.id not in detected_obj_track_ids:
                track.linked_dets.append(Tracklet(timestamp, track.det))

        self.prev_image = image
        self.prev_time = timestamp

        if num_optical_flow_calls > 0:
            tracking_ms = int(1000 * (time.time() - start))
            logging.info(f'Tracking took {tracking_ms}ms, '
                         f'{num_optical_flow_calls} optical flow calls')

        return self.tracks

Create a list of images to work on from the downloaded files.

In [ ]:
def embed_video_file(path: os.PathLike) -> display.HTML:
  """Embeds a file in the notebook as an html tag with a data-url."""
  path = pathlib.Path(path)
  mime, unused_encoding = mimetypes.guess_type(str(path))
  data = path.read_bytes()

  b64 = base64.b64encode(data).decode()
  return display.HTML(
      textwrap.dedent("""
      <video width="640" height="480" controls>
        <source src="data:{mime};base64,{b64}" type="{mime}">
        Your browser does not support the video tag.
      </video>
      """).format(mime=mime, b64=b64))


In [ ]:
tmp_video_path = "tmp_preview.mp4"

filenames = sorted(glob.glob(f"sample_images/{test_sequence_name}/*.jpg"))
img = cv2.imread(filenames[0])
height, width, layers = img.shape
size = (width, height)

video_writer = cv2.VideoWriter(
    filename=tmp_video_path,
    fourcc=cv2.VideoWriter_fourcc(*"MP4V"), 
    fps=15, 
    frameSize=size)
 
for filename in tqdm(filenames):
    img = cv2.imread(filename)
    video_writer.write(img)
cv2.destroyAllWindows()
video_writer.release()

Re-encode the video, and reduce its size (Colab crashes if you try to embed the full size video).

In [ ]:
subprocess.check_call([
    "ffmpeg", "-y", "-i", tmp_video_path,
    "-vf","scale=800:-1",
    "-crf", "18",
    "-preset", "veryfast",
    "-vcodec", "libx264", preview_video_path])

The images you downloaded are frames of a movie showing a top view of a coral reef with crown-of-thorns starfish.  The movie looks like this:

In [ ]:
embed_video_file(preview_video_path)

The goal of the model is to put boxes around all of the starfish.  Each starfish gets its own ID, and that ID will be stable as the camera passes over it.

## Perform the COTS detection inference and tracking.

The detection inference has the following four main steps:
1.    Read all images in the order of image indexes and convert them into uint8 TF tensors (Line 45-54).
2.    Feed the TF image tensors into the model (Line 61) and get the detection output `detections`. In particular, the shape of input tensor is [batch size, height, width, number of channels]. In this demo project, the input shape is [4, 1080, 1920, 3].
3.    The inference output `detections` contains four variables: `num_detections` (the number of detected objects), `detection_boxes` (the coordinates of each COTS object's bounding box), `detection_classes` (the class label of each detected object), `detection_scores` (the confidence score of each detected COTS object).
4.    To track the movement of each detected object across frames, in each frame's detection, the tracker will estimate each tracked COTS object's position if COTS is not detected.


In [ ]:
# Record all the detected COTS objects with the scores equal to or greater than the threshold
threshold = 0.4
_CLASS_ID_TO_LABEL = ('COTS',)
# Create a tracker object
tracker = OpticalFlowTracker(tid=1)
# Record tracking responses from the tracker
detection_result = []
# Record the length of each tracking sequence
track_length_dict = {}

base_time = tf.timestamp()

# Format tracker response, and save it into a new object.
def format_tracker_response(file_path, tracks, seq_length_dict):
    new_track_list = []
    for track in tracks:
        detection_columns = [
            _CLASS_ID_TO_LABEL[track.det.class_id],
            str(track.det.score),
            str(track.id),
            str(len(track.linked_dets)),
            str(round(track.det.x0)),
            str(round(track.det.y0)),
            str(round(track.det.x1 - track.det.x0)),
            str(round(track.det.y1 - track.det.y0))
        ]

        if str(track.id) not in seq_length_dict:
            seq_length_dict[str(track.id)] = len(track.linked_dets)
        else:
            if len(track.linked_dets) > seq_length_dict[str(track.id)]:
                seq_length_dict[str(track.id)] = len(track.linked_dets)
        new_track_list.append({"score":str(round(track.det.score, 3)), "seq_id": str(track.id), "seq_idx": str(len(track.linked_dets)),
                            "x0": round(track.det.x0), "y0": round(track.det.y0), "x1": round(track.det.x1), "y1": round(track.det.y1)})
    return file_path, new_track_list, seq_length_dict

# Read a jpg image and decode it to a uint8 tf tensor.
def parse_image(filename):
    image = tf.io.read_file(filename)
    image = tf.io.decode_jpeg(image)
    return (tf.timestamp(), filename, image)

# Create a data loader
file_list = sorted(glob.glob(f"sample_images/{test_sequence_name}/*.jpg"))
list_ds = tf.data.Dataset.from_tensor_slices(file_list)
images_ds = list_ds.map(parse_image)

# Traverse the dataset with batch size = 1, you cannot change the batch size
for data in tqdm(images_ds.batch(1, drop_remainder=True)):
    # timestamp is used for recording the order of frames
    timestamp, file_path, image = data
    timestamp = (timestamp - base_time) * 1000
    # get detection result
    detections = model_fn(image)
    num_detections = detections['num_detections'].numpy().astype(np.int32)
    detection_boxes = detections['detection_boxes'].numpy()
    detection_classes = detections['detection_classes'].numpy().astype(np.int32)
    detection_scores = detections['detection_scores'].numpy()

    batch_size, img_h, img_w = image.shape[0:3]

    for batch_index in range(batch_size):
        valid_indices = detection_scores[batch_index, :] >= threshold
        classes = detection_classes[batch_index, valid_indices]
        scores = detection_scores[batch_index, valid_indices]
        boxes = detection_boxes[batch_index, valid_indices, :]
        detections = []

        for class_id, score, box in zip(classes, scores, boxes):
            detections.append(
                Detection(
                    class_id=class_id,
                    score=score,
                    x0=box[1] * img_w,
                    y0=box[0] * img_h,
                    x1=box[3] * img_w,
                    y1=box[2] * img_h,
                ))
        # Feed detection results and the corresponding timestamp to the tracker, and then get tracker response
        tracks = tracker.update(image[batch_index].numpy(), detections, timestamp[batch_index])
        base_file_path, track_list, track_length_dict = format_tracker_response(file_path[batch_index].numpy().decode("utf-8"), tracks, track_length_dict)
        detection_result.append((base_file_path, track_list))

# Output the detection results and play the result video
Once the inference is done, we use OpenCV to draw the bounding boxes (Line 9-10) and write the tracked COTS's information (Line 13-20: `COTS ID` `(sequence index/ sequence length)`) on each frame's image. Finally, we combine all frames into a video for visualisation.

In [ ]:
detection_full_video_path = "COTS_detection_full_size.mp4"
detect_video_writer = cv2.VideoWriter(
    filename=detection_full_video_path,
    fourcc=cv2.VideoWriter_fourcc(*"MP4V"), 
    fps=15, 
    frameSize=size)

for file_path, tracks in tqdm(detection_result):
    image = cv2.imread(file_path)
    for track in tracks:
      # Draw the predicted bounding box
      cv2.rectangle(image, (track['x0'], track['y0']),
                (track['x1'], track['y1']),
                (0, 140, 255), thickness=2,)
      # Write the tracked COTS ID, and its corresponding tracking index and tracking sequence length
      cv2.putText(image, f"{track['seq_id']}", (track['x0'], track['y0']-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 140, 255), 2)
      if len(track["seq_id"]) == 1:
          offset = 20
      elif len(track["seq_id"]) == 2:
          offset = 40
      else:
          offset = 60
      cv2.putText(image, 
                  f"({track['seq_idx']}/{track_length_dict[track['seq_id']]})",
                  (track['x0'] + offset, track['y0']-10),
                  cv2.FONT_HERSHEY_SIMPLEX,
                  0.6, (0, 140, 255), 2)
    detect_video_writer.write(image)
cv2.destroyAllWindows()

detect_video_writer.release()

In [ ]:
subprocess.check_call([
    "ffmpeg","-y", "-i", detection_full_video_path,
    "-vf","scale=800:-1",
    "-crf", "18",
    "-preset", "veryfast",
    "-vcodec", "libx264", detection_small_video_path])

In [ ]:
embed_video_file(detection_small_video_path)

The output video is now saved as movie at `detection_full_video_path`. You can download your video by uncommenting the following code.

In [ ]:
#try:
#  from google.colab import files
#  files.download(detection_full_video_path)
#except ImportError:
#  pass